# Packages

In [1]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup 
import pandas as pd 
import numpy as np 
import requests 
import time
from selenium_stealth import stealth
import warnings
warnings.simplefilter("ignore")

# Start time

In [2]:
start_time = time.time()

# Setup

In [3]:
# Information of Setup 
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8,ar;q=0.7"} 
Path = "C:\\Program Files (x86)\\chromedriver.exe"
options = webdriver.ChromeOptions()
# options.add_argument("start-maximized")
options.add_argument("--headless")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(Path,options=options)
stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

# Data Container

In [4]:
## Data Container
Data = {'Name':[], 
        'price':[],
        'link':[],  
        'product_image':[],
        'Product_details':[],
        'tech_info':[],
        'additional_info':[],
        'reviews':[]
        }

# Information Scraper

In [5]:
def get_information(URL):
    elements = 0 
    page = requests.get(URL, headers=headers)
    soup = BeautifulSoup(page.content,'lxml')

    ## Scrape Name
    try :
        Data['Name'].append(soup.find("span",{"id":"productTitle"}).text.strip())
        elements += 1 
    except:
        Data['Name'].append(np.nan)
        
    ## Scrape Price 
    try: 
        Data['price'].append(soup.find("span",{"class":"a-price-whole"}).text.split()[0])
        elements += 1
    except:
        Data['price'].append(np.nan)
        
    ## Scrape Tech_info
    try:    
        Data['tech_info'].append(soup.find("table",{"id":"productDetails_techSpec_section_1"}).text)
        elements += 1
    except:
        Data['tech_info'].append(np.nan)
      
    ## Scrape Additional information
    try: 
        Data['additional_info'] .append(soup.find("table",{"id":"productDetails_detailBullets_sections1"}).text)
        elements += 1
    except: 
        Data['additional_info'] .append(np.nan)
        
    ## Scrape Image 
    try:
        Data['product_image'].append(soup.find("img",{"id":"landingImage"}).attrs['src'])
        elements += 1
    except:
        Data['product_image'].append(np.nan)
        
    ## Scrape Product details
    try:
        Data['Product_details'].append(soup.find("div",{"id":"feature-bullets"}).text)
        elements += 1
    except :
        Data['Product_details'].append(np.nan)
        
    print(f"{elements} Scraped Succefully")

# Reviews Scraper

In [6]:
def get_reviews(URL):
    reviews = ""
    try : 
        driver.get(URL)
        
        ## Find all reviews 
        get_review  = driver.find_element(By.XPATH,'//*[@id="reviews-medley-footer"]/div[2]/a')
        get_review.send_keys(Keys.RETURN)
        
        ## Scrape current page 
        url_ =  driver.current_url
        page = requests.get(url_, headers=headers)
        soup = BeautifulSoup(page.content,'lxml')
        reviews += soup.find("div",{"id":"cm_cr-review_list"}).text.strip()

        c = 0 
        try :
            while c < 20 : 
                c += 1 
                get_review  = driver.find_element(By.XPATH,'//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a')
                get_review.send_keys(Keys.RETURN)
                url_ =  driver.current_url
                page = requests.get(url_, headers=headers)
                soup = BeautifulSoup(page.content,'lxml')
                reviews += soup.find("div",{"id":"cm_cr-review_list"}).text.strip() 
        except :
            pass        
   
    except : 
        pass
        

    if (reviews) : 
        Data['reviews'].append(reviews) 
    else :
        Data['reviews'].append("No reviews Founded")

# Main 

In [7]:
# # Start point 
# MAIN_URL = "https://www.amazon.in/s?k=kitchen+and+dining+products&i=kitchen&sprefix=kitchen+and+d%2Ckitchen%2C167&ref=nb_sb_ss_pltr-ranker-20mins_1_13"
# driver.get(MAIN_URL)
# # All Pages Scrapping 
# page = requests.get(MAIN_URL, headers=headers)
# soup = BeautifulSoup(page.content,'lxml')

# Pages counter 
pages = 0 

counter = 1
for iter_ in range(70,150):
    MAIN_URL = f"https://www.amazon.in/s?k=kitchen+and+dining+products&i=kitchen&page={iter_}&qid=1665815167&sprefix=kitchen+and+d%2Ckitchen%2C167&ref=sr_pg_{iter_}"
    # All Pages Scrapping 
    page = requests.get(MAIN_URL, headers=headers)
    soup = BeautifulSoup(page.content,'lxml')
    page_products_links = soup.find_all('a',{"class":"a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"})
    for url_structure in page_products_links:
        ## Product URL
        half_url = url_structure.attrs['href']
        url = f"https://www.amazon.in/{half_url}" 
        
        ## Save URl 
        Data['link'].append(url)
        print(f"{counter}- ",end="")
        counter += 1
        
        
        get_information(url)
        get_reviews(url)
        time.sleep(1)
    
        
    # print number of page 
    print(f"Page ({iter_}) Scrapped Successfully")

    # Saving_Data 
    dataframe = pd.DataFrame(Data)
    dataframe.to_csv(f"Data({iter_})-HD.csv" , index=False , encoding="utf-8" )

# Say Good bye 
driver.close()

print("--- %s seconds ---" % (time.time() - start_time))

1- 6 Scraped Succefully
2- 6 Scraped Succefully
3- 6 Scraped Succefully
4- 6 Scraped Succefully
5- 6 Scraped Succefully
6- 6 Scraped Succefully
7- 6 Scraped Succefully
8- 6 Scraped Succefully
9- 6 Scraped Succefully
10- 6 Scraped Succefully
11- 6 Scraped Succefully
12- 6 Scraped Succefully
13- 6 Scraped Succefully
14- 6 Scraped Succefully
15- 6 Scraped Succefully
16- 6 Scraped Succefully
17- 6 Scraped Succefully
18- 6 Scraped Succefully
19- 6 Scraped Succefully
20- 6 Scraped Succefully
21- 6 Scraped Succefully
22- 6 Scraped Succefully
23- 6 Scraped Succefully
24- 6 Scraped Succefully
25- 6 Scraped Succefully
26- 6 Scraped Succefully
27- 6 Scraped Succefully
28- 6 Scraped Succefully
29- 6 Scraped Succefully
30- 6 Scraped Succefully
31- 6 Scraped Succefully
32- 6 Scraped Succefully
33- 6 Scraped Succefully
Page (70) Scrapped Successfully
34- 6 Scraped Succefully
35- 6 Scraped Succefully
36- 6 Scraped Succefully
37- 6 Scraped Succefully
38- 6 Scraped Succefully
39- 6 Scraped Succefully
40

309- 6 Scraped Succefully
310- 6 Scraped Succefully
311- 6 Scraped Succefully
312- 6 Scraped Succefully
313- 6 Scraped Succefully
314- 6 Scraped Succefully
315- 6 Scraped Succefully
316- 6 Scraped Succefully
317- 6 Scraped Succefully
318- 6 Scraped Succefully
319- 6 Scraped Succefully
320- 6 Scraped Succefully
321- 6 Scraped Succefully
322- 6 Scraped Succefully
323- 6 Scraped Succefully
324- 6 Scraped Succefully
325- 6 Scraped Succefully
326- 6 Scraped Succefully
327- 6 Scraped Succefully
328- 6 Scraped Succefully
329- 6 Scraped Succefully
330- 6 Scraped Succefully
Page (79) Scrapped Successfully
331- 6 Scraped Succefully
332- 6 Scraped Succefully
333- 6 Scraped Succefully
334- 6 Scraped Succefully
335- 6 Scraped Succefully
336- 6 Scraped Succefully
337- 6 Scraped Succefully
338- 6 Scraped Succefully
339- 6 Scraped Succefully
340- 6 Scraped Succefully
341- 6 Scraped Succefully
342- 6 Scraped Succefully
343- 6 Scraped Succefully
344- 6 Scraped Succefully
345- 6 Scraped Succefully
346- 6

613- 6 Scraped Succefully
614- 6 Scraped Succefully
615- 6 Scraped Succefully
616- 6 Scraped Succefully
617- 6 Scraped Succefully
618- 6 Scraped Succefully
619- 6 Scraped Succefully
620- 6 Scraped Succefully
621- 6 Scraped Succefully
622- 6 Scraped Succefully
623- 6 Scraped Succefully
624- 6 Scraped Succefully
625- 6 Scraped Succefully
626- 6 Scraped Succefully
627- 6 Scraped Succefully
Page (88) Scrapped Successfully
628- 6 Scraped Succefully
629- 6 Scraped Succefully
630- 6 Scraped Succefully
631- 6 Scraped Succefully
632- 6 Scraped Succefully
633- 6 Scraped Succefully
634- 6 Scraped Succefully
635- 6 Scraped Succefully
636- 6 Scraped Succefully
637- 6 Scraped Succefully
638- 4 Scraped Succefully
639- 6 Scraped Succefully
640- 6 Scraped Succefully
641- 6 Scraped Succefully
642- 6 Scraped Succefully
643- 6 Scraped Succefully
644- 6 Scraped Succefully
645- 6 Scraped Succefully
646- 6 Scraped Succefully
647- 6 Scraped Succefully
648- 6 Scraped Succefully
649- 6 Scraped Succefully
650- 6

916- 6 Scraped Succefully
917- 6 Scraped Succefully
918- 6 Scraped Succefully
919- 6 Scraped Succefully
920- 6 Scraped Succefully
921- 6 Scraped Succefully
922- 0 Scraped Succefully
923- 6 Scraped Succefully
924- 0 Scraped Succefully
Page (98) Scrapped Successfully
Page (99) Scrapped Successfully
925- 0 Scraped Succefully
926- 0 Scraped Succefully
927- 4 Scraped Succefully
928- 4 Scraped Succefully
929- 6 Scraped Succefully
930- 6 Scraped Succefully
931- 6 Scraped Succefully
932- 0 Scraped Succefully
933- 0 Scraped Succefully
934- 0 Scraped Succefully
935- 4 Scraped Succefully
936- 0 Scraped Succefully
937- 0 Scraped Succefully
938- 0 Scraped Succefully
939- 0 Scraped Succefully
940- 6 Scraped Succefully
941- 0 Scraped Succefully
942- 0 Scraped Succefully
943- 0 Scraped Succefully
944- 0 Scraped Succefully
945- 4 Scraped Succefully
946- 0 Scraped Succefully
947- 0 Scraped Succefully
948- 4 Scraped Succefully
Page (100) Scrapped Successfully
949- 0 Scraped Succefully
950- 0 Scraped Succ

Page (126) Scrapped Successfully
1189- 4 Scraped Succefully
1190- 0 Scraped Succefully
1191- 0 Scraped Succefully
1192- 4 Scraped Succefully
1193- 4 Scraped Succefully
1194- 0 Scraped Succefully
1195- 0 Scraped Succefully
1196- 0 Scraped Succefully
1197- 4 Scraped Succefully
1198- 0 Scraped Succefully
1199- 4 Scraped Succefully
1200- 0 Scraped Succefully
1201- 4 Scraped Succefully
1202- 0 Scraped Succefully
1203- 0 Scraped Succefully
1204- 0 Scraped Succefully
1205- 0 Scraped Succefully
1206- 0 Scraped Succefully
1207- 4 Scraped Succefully
1208- 0 Scraped Succefully
1209- 0 Scraped Succefully
1210- 4 Scraped Succefully
1211- 0 Scraped Succefully
1212- 0 Scraped Succefully
Page (127) Scrapped Successfully
Page (128) Scrapped Successfully
Page (129) Scrapped Successfully
1213- 0 Scraped Succefully
1214- 4 Scraped Succefully
1215- 0 Scraped Succefully
1216- 4 Scraped Succefully
1217- 0 Scraped Succefully
1218- 4 Scraped Succefully
1219- 0 Scraped Succefully
1220- 4 Scraped Succefully
1221

# End